In [2]:
def dicmaker(): 
    import pandas as pd 
    ilist = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
    namelist = [0,"DP","CD4","CD8","Th1","Th2","Th17","iTreg", "nTreg", "NKT", "CD8-Fli1",11,"Th2-Fli1","DP-Replicate", 14,"Th2 Replicate"]
    datadic = {}
    for m in range(1,len(ilist)+1):
        keyname = "data" +str(m)
        infoloc = "/home/sreedev/Downloads/Data/info" + str(m) + ".txt"
        bedloc = "/home/sreedev/Downloads/Data/bed" + str(m) + ".bedgraph"
        tbloc = "/home/sreedev/Downloads/Data/tb" + str(m) + ".seqspec"

        infodf = pd.read_csv(infoloc,delimiter = "\t",header = None)
        tbdf = pd.read_csv(tbloc,delimiter = "\t", header = None)
        beddf = pd.read_csv(bedloc,delimiter = "\t", header = None)

        datalist = [infodf,tbdf,beddf]
        datadic[keyname] = datalist
    return datadic



In [9]:
def pdmaker(datadic,celltype1,celltype2):
    import re
    pdata = {}
    dickey1 = "data" + str(celltype1)
    infodf1 = datadic[dickey1][0]
    tbdf1 = datadic[dickey1][1]
    dickey2 = "data" + str(celltype2)
    infodf2 = datadic[dickey2][0]
    tbdf2 = datadic[dickey2][1]
    seperators = r":|-"
    for i in range(3,len(infodf1.columns)-1):
        nalist1 = infodf1[i].tolist()
        loclist1 = infodf1[1].tolist()
        matchlist = []
        mvalue = 0
        nvalue = 0
        for n in nalist1:
            mvalue +=1
            if n < 2000:
                nvalue +=1
        for j in range(3,len(infodf2.columns)-1):
            nalist2 = infodf2[j].tolist()
            loclist2 = infodf2[1].tolist()
            kvalue = 0
            xvalue = 0
            for k in range(len(nalist1)):
                location1 = re.split(seperators,loclist1[k])
                for l in range(len(nalist2)):
                    if nalist2[l] < 2000:
                        location2 = re.split(seperators,loclist2[l])
                        if location1[0] == location2[0]:
                            if location1[1] > location2[1] and location1[1] < location2[2]:
                                kvalue +=1
                                continue
                            if location1[2] > location2[1] and location1[2] < location2[2]:
                                kvalue +=1
                                continue
                            if location1[2] == location2[2] and location1[1] == location2[1]:
                                kvalue +=1
                                continue
                if nalist1[k] <2000:
                    for l in range(len(nalist2)):
                        if nalist2[l] < 2000:
                            location2 = re.split(seperators,loclist2[l])
                            if location1[0] == location2[0]:
                                if location1[1] > location2[1] and location1[1] < location2[2]:
                                    xvalue +=1
                                    continue
                                if location1[2] > location2[1] and location1[2] < location2[2]:
                                    xvalue +=1
                                    continue
                                if location1[2] == location2[2] and location1[1] == location2[1]:
                                    xvalue +=1
            pdata[str(i-2) +":"+ str(j-2)] = [mvalue,kvalue,nvalue,xvalue]
    return pdata           

In [10]:
def pvalue(pdata):
    from scipy.stats import hypergeom
    for i in pdata.keys():
        dist = hypergeom(pdata[i][0],pdata[i][1],pdata[i][2])
        x = pdata[i][3]
        pvalue = dist.sf(x-1)
        if pvalue < 0.05:
            return i


In [12]:
data = dicmaker()
pdata = pdmaker(data,1,2)
pvalue(pdata)

'1:3'